In [1]:
import pandas as pd

In [2]:
from nsepy import get_history as gh
import datetime as dt
import yfinance as yf
symbol='TATACOFFEE.NS'
start = dt.datetime(2013,6,1)
end = dt.datetime(2022,2,11)
stk_data = yf.download(symbol, start=start, end=end)

[*********************100%%**********************]  1 of 1 completed


In [3]:
stk_data=stk_data[["Open","High","Low","Close"]]
#stk_data.to_csv("Tatacoffee13_21.csv")

In [4]:
from sklearn.preprocessing import MinMaxScaler
Ms = MinMaxScaler()
data1= Ms.fit_transform(stk_data)
print("Len:",data1.shape)

Len: (2144, 4)


In [5]:
data1=pd.DataFrame(data1,columns=["Open","High","Low","Close"])

In [6]:
training_size = round(len(data1 ) * 0.80)
print(training_size)
X_train=data1[:training_size]
X_test=data1[training_size:]
print("X_train length:",X_train.shape)
print("X_test length:",X_test.shape)
y_train=data1[:training_size]
y_test=data1[training_size:]
print("y_train length:",y_train.shape)
print("y_test length:",y_test.shape)

1715
X_train length: (1715, 4)
X_test length: (429, 4)
y_train length: (1715, 4)
y_test length: (429, 4)


In [7]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
performance={"Model":[],"RMSE":[],"MaPe":[],"Lag":[],"Test":[]}

In [9]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing

def simple_exponential_smoothing(dataset, column_name):
    dataset_seas = dataset[column_name]
    test_obs_seas = 28
    train_seas = dataset_seas[:-test_obs_seas]
    test_seas = dataset_seas[-test_obs_seas:]

    model_seas = SimpleExpSmoothing(train_seas)
    results_seas = model_seas.fit()

    # Forecasting
    pred_seas = results_seas.forecast(steps=28)
    pred_seas.to_csv("ses_forecasted_{}.csv".format(test_obs_seas))

    # Evaluating performance
    from sklearn.metrics import mean_squared_error
    rmse_seas = round(mean_squared_error(test_seas, pred_seas, squared=False), 2)
    from sklearn.metrics import mean_absolute_percentage_error
    mape_seas = mean_absolute_percentage_error(test_seas, pred_seas)

    performance["Model"].append("SES")
    performance["RMSE"].append(rmse_seas)
    performance["MaPe"].append(mape_seas)
    performance["Lag"].append(None)
    performance["Test"].append(test_obs_seas)

    perf_seas = pd.DataFrame(performance)
    return perf_seas, results_seas, pred_seas

In [10]:
column_seas = "Close"
perf_seas, result_seas, pred_seas = simple_exponential_smoothing(data1, column_seas)

In [11]:
data1

,Open,High,Low,Close
0,0.517223,0.485749,0.529877,0.522210
1,0.500522,0.491400,0.530969,0.519394
2,0.498434,0.472236,0.528786,0.517972
3,0.506785,0.472236,0.532251,0.520225
4,0.507411,0.479533,0.527149,0.513788
...,...,...,...,...
2139,0.877349,0.819410,0.900409,0.880633
2140,0.856733,0.807862,0.885130,0.870440
2141,0.848643,0.799017,0.873670,0.865612
2142,0.864301,0.810565,0.892224,0.873927


In [12]:
perf_seas

,Model,RMSE,MaPe,Lag,Test
0,SES,0.02,0.01978,None,28
